<a href="https://colab.research.google.com/github/jiyewise/ML-with-PyTorch-Tutorials/blob/main/Introduction_to_torch_autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tensor

`requires_grad` 속성을 True로 설정하면, 텐서에서 이루어지는 모든 연산들이 추적되고, 계산이 완료된 후 `.backward()` 를 호출하면 모든 gradient를 자동으로 계산할 수 있고, 이 gradient는 텐서의 `.grad` 속성에 누적된다.

텐서에서 이루어지는 연산들이 추적되는 걸 중단하려면, `.detach()` 를 호출해서 연산 기록으로부터 detach, 즉 분리할 수 있다.

### Function

텐서와 function은 서로 연결되어 있고, 연산 과정은 acyclic graph를 생성한다.

각 텐서는 `.grad_fn()` 속성을 가지고 있는데, 이는 텐서를 생성한 function을 참조한다. 단, 사용자가 만든 텐서는 예외 (이 텐서는 사용자가 만들었지, function이 만든 게 아니니까. 이때 `.grad_fn()`은 0이다.)


In [ ]:
# Example 1: Create Tensor
import torch
# x = torch.ones(2, 2, requires_grad=True)
x = torch.tensor([[2, 2], [2, 2]], dtype=torch.float, requires_grad=True)
print(x)

tensor([[2., 2.],
        [2., 2.]], requires_grad=True)


In [ ]:
# Example 1: Tensor에 연산 수행하기
y = x + 2
print(y)
# print grad_fn of y
print(y.grad_fn)

tensor([[4., 4.],
        [4., 4.]], grad_fn=<AddBackward0>)


`y`는 연산의 결과로 만들어진 텐서이기 때문에 `grad_fn`을 가진다. `grad_fn`은 텐서를 만든 function을 참조하는데, 여기서는 덧셈으로 만들어졌으므로 `AddBackward0` function이 참조되었다.

In [ ]:
# Example 1: 다른 연산 수행
z = y*y*3 #3(x+2)^2
out = z.mean()

print(z, out)

tensor([[48., 48.],
        [48., 48.]], grad_fn=<MulBackward0>) tensor(48., grad_fn=<MeanBackward0>)


In [ ]:
# Example 2: .requires_grad_ & detach
a = torch.randn(2,2)
a = ((a+3)/(a-1))
print(a)
print(a.requires_grad)
print("-------requires_grad_ to True--------")
a.requires_grad_(True)
print(a.requires_grad)
b = (a*a).sum()
print(b)
print(b.grad_fn)
print(b.detach())

tensor([[-2.0617, -0.6829],
        [ 6.9749, -0.3832]])
False
-------requires_grad_ to True--------
True
tensor(53.5131, grad_fn=<SumBackward0>)
tensor(53.5131)


### Gradient
도함수를 계산하기 위해서는 텐서의 `.backward()` 를 call 하면 된다.
gradient를 계산하고자 하는 텐서가 스칼라라면 (변수가 1개밖에 없다면) `backward`에 따로 텐서 인자를 지정해줄 필요가 없다. 그러나 텐서가 여러 개의 변수를 가질 때는 텐서의 인자를 지정해 주어야 한다. Example 3 참고.

### Vector-Jacobian
vector-jacobian에 대한 설명은 [튜토리얼 링크](https://tutorials.pytorch.kr/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py) 참고.
결과적으로 vector-jacobian은 x vector 즉 (x1, x2, ... xn)에 대한 함수 l의 기울기가 됨을 알 수 있다. 
`torch.autograd`는 이러한 vector-jacobian 곱을 계산하는 엔진.

* [링크](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)에서 가져온 설명인데 가장 잘 이해되는 설명이라 붙여놓음.
"If `x` is a Tensor that has `x.requires_grad=True` then `x.grad` is another Tensor holding the gradient of `x` with respect to some scalar value."


In [ ]:
# Example 3: .backward() when tensor is a scalar
# out.backward()
# print(x.grad) # recall example 1

# Example 3: vector-jacobian by torch.autograd
x = torch.ones(3, requires_grad = True)
y = x * 2
print(x)
while y.data.norm() < 1000:
  y = y * 2
print(y)

v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v) 
# y.backward(x) 
print(x.grad) # y.grad would not work as it is not a leaf tensor 

# Example 3: scalar prod. (when tensor is a scalar) 
print("---------Scalar Prod-----------")
x2 = torch.ones(3, requires_grad = True)
y2 = x2 * 2
print(x2)
while y2.data.norm() < 1000:
  y2 = y2 * 2
print(y2)

out = y2.mean()
out.backward() # default: gradient calculation on values of x
# out.backward(torch.tensor(0.1, dtype=float)) # 0.1 is passed to the gradient
print(x2.grad) 

tensor([1., 1., 1.], requires_grad=True)
tensor([1024., 1024., 1024.], grad_fn=<MulBackward0>)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])
---------Scalar Prod-----------
tensor([1., 1., 1.], requires_grad=True)
tensor([1024., 1024., 1024.], grad_fn=<MulBackward0>)
tensor([341.3333, 341.3333, 341.3333])


In [ ]:
# Example 4: torch.no_grad() & detach
print(x.requires_grad)
print((x**2).requires_grad)

# wrap the code block with no_grad
with torch.no_grad():
  print((x**2).requires_grad)

# detach: detach creates a new tensor with same content but different require_grad
y = x.detach()
print(y.requires_grad) # different require_grad
print(x.eq(y).all()) # same value

True
True
False
False
tensor(True)
